# Brief Idea of Futures Grid

The futures grid bot helps traders buy low and sell high, making profits by matching orders. When a buy order is executed, a new sell order is placed above it. Conversely, when a sell order is executed, a new buy order is placed underneath it.


The bot buys low and sells high only within a certain price range. If the price goes beyond its limits, the bot stops trading until it returns to the specified range.
While a bot offers potential benefits, futures trading comes with risks. The use of leverage can lead to significant losses in the event of unfavorable market conditions. In addition, if the margin falls below a predetermined level, liquidation may begin.

 Futures trading is based on a commitment to buy or sell a specific asset (such as a cryptocurrency) at a fixed price, but the actual transaction occurs on a set date in the future.

# Import Libraries

In [8]:
import ccxt
import pandas as pd ## 
import numpy as np
from datetime import datetime, timedelta ## 
import time ## 
# import matplotlib.pyplot as plt

# Getting Data from Binance for BTC

In [9]:
# Use your Binance API credentials
exchange = ccxt.binance({
    'apiKey': 'rKI3q7RZSOwaKmtEdpJr8gzW2ck32ZXVAUWhwajXU88pHlrm0cnKqyHWy2n2gbRJ',
    'secret': 'RfA74xF2DloTmJcHSVsIMnpk7YbHhfI4iV1SVv8NIR1pHf1rnCl8n14hfNSWXTiu',
    'options': {'defaultType': 'future'}  # Ensure futures trading mode
})

# Fetch the order book for a symbol (e.g., 'BTC/USDT')
order_book = exchange.fetch_order_book('BTC/USDT')

# Creating the Parameters for the Bot

### 
Arithmetic: Each grid has an equal price difference. Better for Low-volatility assets

Geometric: Each grid has an equal price difference ratio. Better for High-volatility assets

In [10]:
# Grid Trading Parameters
symbol = 'BTC/USDT'  # Trading pair
grid_levels = 10  # Number of grid orders
grid_spacing = 50  # Price gap between grid orders
order_size = 0.01  # Order size in BTC
leverage = 1 # Leverage multiplier
grid_type = 'arithmetic'  # or 'geometric'

In [11]:
# Set leverage properly
exchange.set_leverage(leverage, symbol)

{'symbol': 'BTCUSDT', 'leverage': '1', 'maxNotionalValue': '1800000000'}

In [12]:
# Get initial price
price_data = exchange.fetch_ticker(symbol)
initial_price = price_data['last']
print('Initial Price: ', initial_price)
# Generate grid price levels
grid_prices = [initial_price + (i - grid_levels // 2) * grid_spacing for i in range(grid_levels)]
print('Grid Prices: ', grid_prices)

Initial Price:  96915.1
Grid Prices:  [96665.1, 96715.1, 96765.1, 96815.1, 96865.1, 96915.1, 96965.1, 97015.1, 97065.1, 97115.1]


In [13]:
# Initialize Binance exchange (without API keys for public data)
exchange = ccxt.binance()

# Fetch historical data (Example: BTC/USDT, 1-minute timeframe)
bars = exchange.fetch_ohlcv('BTC/USDT', timeframe='1m', limit=500)

# Convert to DataFrame
df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Convert timestamp to readable date
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Save to CSV file
# df.to_csv('binance_data.csv', index=False)

df = pd.read_csv("binance_data.csv")

# Display the first and last few rows
print((df.head()), "\n\n")
print(df.tail())

             timestamp      open      high       low     close    volume
0  2025-02-11 06:43:00  98312.99  98336.79  98312.98  98329.99   8.74084
1  2025-02-11 06:44:00  98329.99  98346.58  98329.99  98346.57  10.12839
2  2025-02-11 06:45:00  98346.58  98388.89  98300.00  98300.00  29.89419
3  2025-02-11 06:46:00  98300.01  98332.01  98288.04  98317.50  13.23001
4  2025-02-11 06:47:00  98317.50  98336.43  98300.00  98300.00  10.06472 


               timestamp      open      high       low     close    volume
495  2025-02-11 14:58:00  96992.03  97015.84  96992.03  96996.10   9.56111
496  2025-02-11 14:59:00  96996.10  97000.06  96962.00  96962.03   5.03029
497  2025-02-11 15:00:00  96962.04  97078.29  96912.65  97045.75  45.29869
498  2025-02-11 15:01:00  97045.74  97130.67  97045.74  97130.67  12.14271
499  2025-02-11 15:02:00  97130.67  97247.09  97116.32  97214.74  14.34648


## Hidden Parameters

In [14]:
# Hidden parameters validation
def validate_grid_parameters(initial_price, grid_levels, grid_spacing, leverage):
    # 1. Price Range
    MIN_PRICE = 556.8
    MAX_PRICE = 4529764
    # 2. Number of Grids
    MIN_GRIDS = 2
    MAX_GRIDS = 169
    # 3. Investment
     # the investment is the margin amount transferred from the futures account 
     # to create the grid. The total investment = investment * leverage. 
    MIN_LEVERAGE = 1
    MAX_LEVERAGE = 1
    # 4. Choose 'arithmetic' or 'geometric'
    grid_type = 'geometric' 

    if initial_price - (grid_levels * grid_spacing) < MIN_PRICE:
        raise ValueError("Grid buy orders exceed Binance's minimum price limit.")
    if initial_price + (grid_levels * grid_spacing) > MAX_PRICE:
        raise ValueError("Grid sell orders exceed Binance's maximum price limit.")
    if not (MIN_GRIDS <= grid_levels <= MAX_GRIDS):
        raise ValueError(f"Number of grid levels must be between {MIN_GRIDS} and {MAX_GRIDS}.")
    if not (MIN_LEVERAGE <= leverage <= MAX_LEVERAGE):
        raise ValueError(f"Leverage must be between {MIN_LEVERAGE}x and {MAX_LEVERAGE}x.")

    print("Hidden parameters validated successfully.")

# Validate parameters before trading
try:
    validate_grid_parameters(initial_price, grid_levels, grid_spacing, leverage)
    print("Proceeding to buy and sell...")
    last_trade = None
except ValueError as e:
    print("Validation Error:", e)

Hidden parameters validated successfully.
Proceeding to buy and sell...


In [15]:
# # Fetch current BTC/USDT price
# ticker = exchange.fetch_ticker('BTC/USDT')
# current_price = ticker['last']
# print(f"Current BTC Price: {current_price}")

# # Generate dynamic grid prices (±1% steps)
# grid_step = 0.01  # 1% per step
# num_grids = 5     # Number of grid levels

# grid_prices = sorted([
#     round(current_price * (1 + grid_step * i), 2) for i in range(-num_grids, num_grids + 1)
# ])

# print(f"Grid Prices: {grid_prices}")

# # Load historical data
# df['time'] = pd.to_datetime(df['timestamp'])
# df.set_index('time', inplace=True)

# # Track last trade direction
# last_trade = None  

# # Simulate trading
# for index, row in df.iterrows():
#     price = row['close']

#     for grid_price in grid_prices:
#         if abs(price - grid_price) <= 10:  # Allow small variation
            
#             if last_trade == "SELL" or last_trade is None:
#                 print(f"🟢 BUY at {price} on {index}")
#                 last_trade = "BUY"
            
#             elif last_trade == "BUY":
#                 print(f"🔴 SELL at {price} on {index}")
#                 last_trade = "SELL"


### 
Taker: u place a market order for a coin and then pay a taker fee since you're taking liquidity out of the market because the order is immediate. 

Maker: u place a limit order for a specific price that doesn't immediately find a matching order.

In [16]:
# Initialize tracking variables
completed_trades = 0
total_profit = 0
trades_history = []
binance_fees = {'maker': 0.0002, 'taker': 0.0005}  # 0.02%, 0.05%


# Fetch current BTC/USDT price
ticker = exchange.fetch_ticker('BTC/USDT')
current_price = ticker['last']
print(f"Current BTC Price: {current_price}")

# Generate grid prices
if grid_type == 'arithmetic':
    grid_prices = sorted([
        round(current_price + (grid_spacing * i), 2) 
        for i in range(-grid_levels // 2, grid_levels // 2 + 1)
    ])
else:  # geometric
    ratio = 1.936204  # Apply the provided ratio
    grid_prices = sorted([
        round(current_price * (ratio ** i), 2) 
        for i in range(-grid_levels // 2, grid_levels // 2 + 1)
    ])

print(f"Grid Prices: {grid_prices}")

# Load historical data
df['time'] = pd.to_datetime(df['timestamp'])  # Convert timestamps to datetime
df.set_index('time', inplace=True)  # Set time as index for easy lookup

# Track last trade direction and price
last_trade = None
last_trade_price = None

# Simulate trading
for index, row in df.iterrows():  # Loop through historical data
    price = row['close']  # Get the closing price of each time period

    for grid_price in grid_prices:
        if abs(price - grid_price) <= 10:  # Allow small variation
            effective_size = order_size * leverage  # Apply leverage to position size
            
            if last_trade == "SELL" or last_trade is None:
                print(f"🟢 BUY {effective_size} {symbol} at {price} on {index}")
                last_trade = "BUY"
                last_trade_price = price
                
                trades_history.append({
                    'timestamp': index,
                    'type': 'BUY',
                    'price': price,
                    'grid_price': grid_price,
                    'size': effective_size
                })
                
            elif last_trade == "BUY":
                print(f"🔴 SELL {effective_size} {symbol} at {price} on {index}")
                
                # Calculate profit for this completed grid
                buy_price = last_trade_price
                sell_price = price
                
                # Calculate fees (considering leverage)
                #buy_fee = buy_price * effective_size * binance_fees['maker']
                #sell_fee = sell_price * effective_size * binance_fees['maker']
                
                # Calculate profit (with leverage)
                trade_profit = (sell_price - buy_price) * effective_size
                total_profit += trade_profit
                completed_trades += 1
                
                trades_history.append({
                    'timestamp': index,
                    'type': 'SELL',
                    'price': price,
                    'grid_price': grid_price,
                    'size': effective_size,
                    'profit': trade_profit
                })
                
                print(f"Trade {completed_trades}: " \
                      f"Buy at {buy_price:.2f}, Sell at {sell_price:.2f}, " \
                      f"Profit: {trade_profit:.6f} USDT (with {leverage}x leverage)")
                
                last_trade = "SELL"
                last_trade_price = None

    # Update grid prices periodically based on current price
    if grid_type == 'arithmetic':
        grid_prices = sorted([
            round(price + (grid_spacing * i), 2) 
            for i in range(-grid_levels // 2, grid_levels // 2 + 1)
        ])
    else:  # geometric
        grid_prices = sorted([
            round(price * (ratio ** i), 2) 
            for i in range(-grid_levels // 2, grid_levels // 2 + 1)
        ])

# Print Final Trading Summary
if completed_trades > 0:
    avg_profit_per_grid = total_profit / completed_trades
    print("\nTrading Summary:")
    print(f"Symbol: {symbol}")
    print(f"Leverage: {leverage}x")
    print(f"Total Profit: {total_profit:.6f} USDT")
    print(f"Completed Trades: {completed_trades}")
    print(f"Average Profit per Grid: {avg_profit_per_grid:.6f} USDT")
    print(f"Profit Percentage: {(avg_profit_per_grid / current_price * 100):.4f}%")
    print(f"Effective Order Size: {order_size * leverage} {symbol.split('/')[0]}")

Current BTC Price: 96968.9
Grid Prices: [96718.9, 96768.9, 96818.9, 96868.9, 96918.9, 96968.9, 97018.9, 97068.9, 97118.9, 97168.9, 97218.9]
🟢 BUY 0.01 BTC/USDT at 98300.0 on 2025-02-11 06:45:00
🔴 SELL 0.01 BTC/USDT at 98460.32 on 2025-02-11 06:52:00
Trade 1: Buy at 98300.00, Sell at 98460.32, Profit: 1.603200 USDT (with 1x leverage)
🟢 BUY 0.01 BTC/USDT at 98424.71 on 2025-02-11 06:56:00
🔴 SELL 0.01 BTC/USDT at 98442.83 on 2025-02-11 06:58:00
Trade 2: Buy at 98424.71, Sell at 98442.83, Profit: 0.181200 USDT (with 1x leverage)
🟢 BUY 0.01 BTC/USDT at 98350.9 on 2025-02-11 07:01:00
🔴 SELL 0.01 BTC/USDT at 98384.88 on 2025-02-11 07:03:00
Trade 3: Buy at 98350.90, Sell at 98384.88, Profit: 0.339800 USDT (with 1x leverage)
🟢 BUY 0.01 BTC/USDT at 98376.13 on 2025-02-11 07:04:00
🔴 SELL 0.01 BTC/USDT at 98421.71 on 2025-02-11 07:05:00
Trade 4: Buy at 98376.13, Sell at 98421.71, Profit: 0.455800 USDT (with 1x leverage)
🟢 BUY 0.01 BTC/USDT at 98353.17 on 2025-02-11 07:07:00
🔴 SELL 0.01 BTC/USDT at

In [17]:
# # calculating Binance geometric ratio based on the numbers auto-fill-in
# def calculate_geometric_ratio(total_range, grid_levels):
#     """
#     Calculate the geometric ratio for grid trading.

#     :param total_range: The total price range as a percentage (e.g., 1.05 for 5% range)
#     :param grid_levels: The total number of grid levels
#     :return: The calculated ratio
#     """
#     return np.exp(np.log(total_range) / (grid_levels // 2))

# # Example usage:
# total_range = 20149.6  # 5% range
# grid_levels = 30  # Total grid levels
# ratio = calculate_geometric_ratio(total_range, grid_levels)

# print(f"Geometric Ratio: {ratio:.6f}")
